In [0]:
%run ./Idempotent-Kafka-to-Bronze

In [0]:
#dbutils.fs.rm("/FileStore/test/checkpoint/invoices", True)

In [0]:
class kafkaToBronzeTestSuite():
  def __init__(self):
    self.base_data_dir = "/FileStore/test"

  def cleanTests(self):
    print(f"Starting Cleanup...", end='')
    spark.sql("drop table if exists invoices_bz")
    dbutils.fs.rm("/user/hive/warehouse/invoices_bz", True)
    dbutils.fs.rm(f"{self.base_data_dir}/checkpoint/invoices_bz", True)
    print("Done")

  def assertResult(self, expected_count):
    print(f"Starting validation...", end='')
    actual_count = spark.sql("select count(*) from invoices_bz").collect()[0][0]
    assert expected_count == actual_count, f"Test failed! actual count is {actual_count}"
    print("Done")

  def waitForMicroBatch(self, sleep=15):
    import time
    print(f"Waiting for {sleep} seconds...", end='')
    time.sleep(sleep)
    print("Done.")    

  def runTests(self):
    self.cleanTests()
    bzStream = Bronze()

    value_schema = bzStream.getSchema()
    spark.sql(f"CREATE TABLE invoices_bz (key STRING, value STRUCT<{value_schema}>, topic STRING, timestamp TIMESTAMP)")

    print("Testing Scenario - Start from beginning on a new checkpoint...") 
    bzQuery = bzStream.process()
    self.waitForMicroBatch() 
    bzQuery.stop()       
    self.assertResult(29)
    print("Validation passed.\n")

    print("Testing Scenarion - Restart from where it stopped on the same checkpoint...")
    bzQuery = bzStream.process()
    self.waitForMicroBatch()
    bzQuery.stop()
    self.assertResult(29)
    print("Validation passed.\n")

In [0]:
ts = kafkaToBronzeTestSuite()
ts.runTests()

Starting Cleanup...Done
Testing Scenario - Start from beginning on a new checkpoint...
Starting Bronze Stream ...
Done.
Waiting for 15 seconds...Done.
Starting validation...Done
Validation passed.

Testing Scenarion - Restart from where it stopped on the same checkpoint...
Starting Bronze Stream ...
Done.
Waiting for 15 seconds...Done.
Starting validation...Done
Validation passed.

